## Step 0: Setup Spark

### Configure Spark for Your Notebook
* You may need to Reconnect and/or Restart the Kernel to pick up changes.
* This examples uses the local Spark Master `--master local[1]`
* In production, you would use the PipelineIO Spark Master `--master spark://apachespark-master-2-1-0:7077`

In [ ]:
import os

master = '--master local[1]'
#master = '--master spark://apachespark-master-2-1-0:7077'
conf = '--conf spark.cores.max=1 --conf spark.executor.memory=512m'
packages = '--packages com.amazonaws:aws-java-sdk:1.7.4,org.apache.hadoop:hadoop-aws:2.7.1'
jars = '--jars /root/lib/jpmml-sparkml-package-1.0-SNAPSHOT.jar'
py_files = '--py-files /root/lib/jpmml.py'

os.environ['PYSPARK_SUBMIT_ARGS'] = master \
  + ' ' + conf \
  + ' ' + packages \
  + ' ' + jars \
  + ' ' + py_files \
  + ' ' + 'pyspark-shell'

print(os.environ['PYSPARK_SUBMIT_ARGS'])

### Import Spark Libraries

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.regression import LinearRegression

### Create Spark Session
This may take a minute or two.  Please be patient.

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

## Step 1: Load Training Data into Spark Cluster

### Read Data from Public S3 Bucket
* AWS credentials are not needed.
* We're asking Spark to infer the schema
* The data has a header
* Using `bzip2` because it's a splittable compression file format

In [ ]:
df = spark.read.format("csv") \
    .option("inferSchema", "true") \
    .option("header", "true") \
    .load("s3a://datapalooza/airbnb/airbnb.csv.bz2")

print(df.head())

In [ ]:
print(df.count())

### Clean, Filter, and Summarize the Data

In [ ]:
df_filtered = df.filter("price >= 50 AND price <= 750 AND bathrooms > 0.0 AND bedrooms is not null")

df_filtered.registerTempTable("df_filtered")

df_final = spark.sql("""
    select
        id,
        city,
        case when state in('NY', 'CA', 'London', 'Berlin', 'TX' ,'IL', 'OR', 'DC', 'WA')
            then state
            else 'Other'
        end as state,
        space,
        cast(price as double) as price,
        cast(bathrooms as double) as bathrooms,
        cast(bedrooms as double) as bedrooms,
        room_type,
        host_is_super_host,
        cancellation_policy,
        cast(case when security_deposit is null
            then 0.0
            else security_deposit
        end as double) as security_deposit,
        price_per_bedroom,
        cast(case when number_of_reviews is null
            then 0.0
            else number_of_reviews
        end as double) as number_of_reviews,
        cast(case when extra_people is null
            then 0.0
            else extra_people
        end as double) as extra_people,
        instant_bookable,
        cast(case when cleaning_fee is null
            then 0.0
            else cleaning_fee
        end as double) as cleaning_fee,
        cast(case when review_scores_rating is null
            then 80.0
            else review_scores_rating
        end as double) as review_scores_rating,
        cast(case when square_feet is not null and square_feet > 100
            then square_feet
            when (square_feet is null or square_feet <=100) and (bedrooms is null or bedrooms = 0)
            then 350.0
            else 380 * bedrooms
        end as double) as square_feet
    from df_filtered
""").persist()

df_final.registerTempTable("df_final")

df_final.select("square_feet", "price", "bedrooms", "bathrooms", "cleaning_fee").describe().show()

In [ ]:
print(df_final.count())

In [ ]:
print(df_final.schema)

In [ ]:
# Most popular cities

spark.sql("""
    select 
        state,
        count(*) as ct,
        avg(price) as avg_price,
        max(price) as max_price
    from df_final
    group by state
    order by count(*) desc
""").show()

In [ ]:
# Most expensive popular cities

spark.sql("""
    select 
        city,
        count(*) as ct,
        avg(price) as avg_price,
        max(price) as max_price
    from df_final
    group by city
    order by avg(price) desc
""").filter("ct > 25").show()

## Step 2: Feature Engineering

In [ ]:
continuous_features = ["bathrooms", \
                       "bedrooms", \
                       "security_deposit", \
                       "cleaning_fee", \
                       "extra_people", \
                       "number_of_reviews", \
                       "square_feet", \
                       "review_scores_rating"]

categorical_features = ["room_type", \
                        "host_is_super_host", \
                        "cancellation_policy", \
                        "instant_bookable", \
                        "state"]

### Continuous Features

In [ ]:
continuous_feature_assembler = VectorAssembler(inputCols=continuous_features, outputCol="unscaled_continuous_features")

continuous_feature_scaler = StandardScaler(inputCol="unscaled_continuous_features", outputCol="scaled_continuous_features", \
                                           withStd=True, withMean=False)

### Categorical Features

In [ ]:
categorical_feature_indexers = [StringIndexer(inputCol=x, \
                                              outputCol="{}_index".format(x)) \
                                for x in categorical_features]

categorical_feature_one_hot_encoders = [OneHotEncoder(inputCol=x.getOutputCol(), \
                                                      outputCol="oh_encoder_{}".format(x.getOutputCol() )) \
                                        for x in categorical_feature_indexers]

### Assemble Continuous and Categorical Features

In [ ]:
feature_cols_lr = [x.getOutputCol() \
                   for x in categorical_feature_one_hot_encoders]
feature_cols_lr.append("scaled_continuous_features")

feature_assembler_lr = VectorAssembler(inputCols=feature_cols_lr, \
                                       outputCol="features_lr")

## Step 3: Train the Pipeline

In [ ]:
linear_regression = LinearRegression(featuresCol="features_lr", \
                                     labelCol="price", \
                                     predictionCol="price_prediction", \
                                     maxIter=10, \
                                     regParam=0.3, \
                                     elasticNetParam=0.8)

estimators_lr = \
  [continuous_feature_assembler, continuous_feature_scaler] \
  + categorical_feature_indexers + categorical_feature_one_hot_encoders \
  + [feature_assembler_lr] + [linear_regression]

pipeline = Pipeline(stages=estimators_lr)

pipeline_model = pipeline.fit(df_final)

print(pipeline_model)

## Step 4:  Export the Pipeline Model

In [ ]:
from jpmml import toPMMLBytes

model = toPMMLBytes(spark, df_final, pipeline_model)

with open('airbnb.model', 'wb') as fh:
    fh.write(model)

## Step 5:  Deploy the Pipeline Model

### Option 1 of 2:  Deploy with PipelineIO CLI!

In [ ]:
%%bash

pip install -q pio-cli==0.40

Configure CLI for Model Deployment

In [ ]:
%%bash

pio init-model --model-server-url=http://prediction-jvm.demo.pipeline.io --model-type=pmml \
    --model-namespace=cli --model-name=airbnb

In [ ]:
%%bash

pio deploy --model-version=v0 airbnb.model

### Deployment Option 2:  REST API

In [ ]:
import requests

deploy_url = 'http://prediction-jvm.demo.pipeline.io/api/v1/model/deploy/pmml/rest/airbnb/v0'

files = {'file': open('airbnb.model', 'rb')}

response = requests.post(deploy_url, files=files)

print("Success! %s" % response.text)

## Step 6:  Predict With Deployed Pipeline Model

### Setup Prediction Inputs

In [ ]:
import json

data = {"bathrooms":2.0, 
        "bedrooms":2.0, 
        "security_deposit":175.00, 
        "cleaning_fee":25.0, 
        "extra_people":1.0, 
        "number_of_reviews": 2.0, 
        "square_feet": 250.0, 
        "review_scores_rating": 2.0, 
        "room_type": "Entire home/apt", 
        "host_is_super_host": "0.0", 
        "cancellation_policy": "flexible", 
        "instant_bookable": "1.0", 
        "state": "CA" }

json_data = json.dumps(data)

with open('airbnb-predict-inputs.json', 'wt') as fh:
    fh.write(json_data)

### Predict with CLI
Note:  Run again if you see a fallback on the first try. (bug)

In [ ]:
%%bash

pio predict --model-version=v0 \
            --model-input-filename=airbnb-predict-inputs.json

### Predict with REST

In [ ]:
import json

# Note:  You may need to run this twice.
#        A fallback will trigger the first time. (Bug)
predict_url = 'http://prediction-jvm.demo.pipeline.io/api/v1/model/predict/pmml/rest/airbnb/v0'

headers = {'content-type': 'application/json'}

response = requests.post(predict_url, 
                         data=json_data, 
                         headers=headers)

print(response.text)

## Step 7:  Monitor Model Servers through Dashboards

### Fallbacks and Circuit Breaker [Dashboard](http://hystrix.demo.pipeline.io/hystrix-dashboard/monitor/monitor.html?streams=%5B%7B%22name%22%3A%22Model%20Servers%22%2C%22stream%22%3A%22http%3A%2F%2Fturbine.demo.pipeline.io%2Fturbine.stream%22%2C%22auth%22%3A%22%22%2C%22delay%22%3A%22%22%7D%5D)

In [ ]:
%%html

<iframe width=800 height=600 src="http://hystrix.demo.pipeline.io/hystrix-dashboard/monitor/monitor.html?streams=%5B%7B%22name%22%3A%22Model%20Servers%22%2C%22stream%22%3A%22http%3A%2F%2Fturbine.demo.pipeline.io%2Fturbine.stream%22%2C%22auth%22%3A%22%22%2C%22delay%22%3A%22%22%7D%5D"></iframe>

### Grafana Prediction Metrics [Dashboard](http://grafana.demo.pipeline.io)

In [ ]:
%%html

<iframe width=800 height=600 src="http://grafana.demo.pipeline.io"></iframe>